# Pip Installs

In [18]:
%%capture

!pip install librosa
!pip install tensorflow
!pip install numpy==1.23.5
!pip install pysoundfile

Tensor flow does not work with Numpy 2.0 onward so make sure that 

# Imports

Below are the necessary imports 

In [134]:
#Imports
from pydub import AudioSegment as audio
import os
import numpy as np
import librosa
#Tensor Flow needs numpy at 1.23.5
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import re 
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# File Creation

### Creating File Paths

In [101]:
unprocessed_data_dir = os.path.abspath('') + "/Unprocessed_Wav_Files"
proccesed_data_dir = os.path.abspath('') + "/Processed_Wav_Files"

Steps
- go through all files in the unprocessed wav files
- break up all files so they are split by 1 second
- use regex to get the file name minus the .wav at the end to make new 1 second wav files
- save each of the files into the processed wav folder

Resample the unproccessed audio files for speaker recognition. This is needed for tensorflow and keras

In [49]:
#This was a test for the fixing the audio with a single thing

"""
f_path = os.path.join(unproccessed_data_dir, "addisonWilson40173Clip1.wav")
arr, hz = librosa.load(f_path, sr = None)
new_f = librosa.resample(arr, orig_sr= hz, target_sr= 16000)
sf.write(os.path.join(unproccessed_data_dir,"new.wav"),new_f, 16000)
"""


'\nf_path = os.path.join(unproccessed_data_dir, "addisonWilson40173Clip1.wav")\narr, hz = librosa.load(f_path, sr = None)\nnew_f = librosa.resample(arr, orig_sr= hz, target_sr= 16000)\nsf.write(os.path.join(unproccessed_data_dir,"new.wav"),new_f, 16000)\n'

### Changes the Unporcessed_Wav_Files into 16000 hz

This is necessary for speaker recognition model training and feature extractions

In [102]:
for file in os.listdir(unprocessed_data_dir):
    f_path = os.path.join(unprocessed_data_dir, file)
    arr, hz = librosa.load(f_path,sr = None)
    if hz != 16000:
        new_f = librosa.resample(arr, orig_sr = hz, target_sr = 16000)
        sf.write(f_path, new_f, 16000)

This is a test

In [103]:
f_path = os.path.join(unprocessed_data_dir, "addisonWilson40173Clip1.wav")
arr, hz = librosa.load(f_path, sr = None)
print(hz)

16000


### Separates the files into 1 second intervals

In [129]:
for filename in os.listdir(unprocessed_data_dir):
    if filename.endswith(".wav"):
        hearing = audio.from_wav(os.path.join(unprocessed_data_dir,filename))
        speaker = filename.removesuffix(".wav")
        #name of congress person for processed_wav_files folder
        congress_name = speaker[:re.search(r"\d+",speaker).start()] 
        #start duration in milliseconds for audio recording start
        t1 = 0
        #end duration in milliseconds for audio recording start
        t2 = 1000
        #second of the clip
        second = 1
        total_time = hearing.duration_seconds*1000
        # Goes through all of the large audio files, splits them into 1 second audio files
        # and saves to the speaker name folder
        while t2 <= total_time:
            new_audio = hearing[t1:t2]
            #arr, hz = librosa.load(new_audio, sr = None)
            try:
                os.makedirs(f"{proccesed_data_dir}/{congress_name}")
                new_audio.export(f"{proccesed_data_dir}/{congress_name}/{speaker}_{second}.wav", format = "wav");
                #new_audio.export(f"{proccesed_data_dir}/{congress_name}/{speaker}_{second}.wav", format = "wav", parameters=["-ar", "16000"]);
            except:
                #new_audio.export(f"{proccesed_data_dir}/{congress_name}/{speaker}_{second}.wav", format = "wav", parameters=["-ar", "16000"]);
                new_audio.export(f"{proccesed_data_dir}/{congress_name}/{speaker}_{second}.wav", format = "wav");
            t1 += 1000
            t2 += 1000
            second += 1

Check the Hz of a split file

In [130]:
a,b = librosa.load(os.path.abspath('')  + "/Processed_Wav_Files/addisonWilson/addisonWilson40173Clip1_2.wav", sr = None)
print(b)

16000


### Feature Extraction

In [157]:
def feature_extract (data_dir):
    features = []
    labels = []
    for foldr in os.listdir(data_dir):
        if foldr != ".DS_Store":
            for file in os.listdir(f"{data_dir}/{foldr}"):
                arr, hz = librosa.load(os.path.join(data_dir,foldr,file), sr = None, duration =  1)
                mfccs = librosa.feature.mfcc(y = arr, sr = hz, n_mfcc = 13)
                mfccs = StandardScaler().fit_transform(mfccs)
                features.append(mfccs.T)
                labels.append(foldr)       
    return np.array(features), np.array(labels)           
features, labels = feature_extract(proccesed_data_dir)

### Feature Visualization

In [165]:
for feature_set, label in zip(features, (x for x in len(labels) if x%100 == 0)):
    print(f"Label: {label}, Features Shape: {feature_set.shape}")
    print(feature_set)  # Optionally print the actual features
    print("-" * 40)  # Separator for readability

TypeError: 'int' object is not iterable